In [ ]:
import time
import xarray as xr
import numpy as np
from xcube.core.geocoded import reproject_dataset, compute_output_geom

In [ ]:
path = 'C:\\Users\\Norman\\Downloads\\S3B_OL_1_EFR____20190728T103451_20190728T103751_20190729T141105_0179_028_108_1800_LN1_O_NT_002.SEN3'

In [ ]:
vars = dict()
for f in ('qualityFlags.nc', 'geo_coordinates.nc', 'Oa06_radiance.nc', 'Oa13_radiance.nc', 'Oa20_radiance.nc'): 
    ds = xr.open_dataset(path + '\\' + f)
    vars.update(ds.data_vars)
src_ds = xr.Dataset(vars)
src_ds

In [ ]:
src_ds['longitude'] = xr.DataArray(src_ds.longitude.values, dims=('rows', 'columns'))
src_ds['latitude'] = xr.DataArray(src_ds.latitude.values, dims=('rows', 'columns'))
src_ds

In [ ]:
compute_output_geom(src_ds, xy_names=('longitude', 'latitude'))

In [ ]:
src_ds.Oa13_radiance.plot.imshow(figsize=(16,16), vmax=40, cmap='Greys_r')

In [ ]:
src_ds.longitude.diff('rows').plot.imshow(figsize=(12,12), vmin=-0.005, vmax=0.005)

In [ ]:
src_ds.longitude.diff('columns').plot.imshow(figsize=(12,12), vmin=-0.01, vmax=0.01)

In [ ]:
src_ds.latitude.diff('rows').plot.imshow(figsize=(12,12), vmin=-0.003, vmax=-0.002)

In [ ]:
src_ds.latitude.diff('columns').plot.imshow(figsize=(12,12), vmin=-0.002, vmax=0.0)

In [ ]:
col_diff = np.sqrt(np.square(src_ds.longitude.diff('columns')) + np.square(src_ds.latitude.diff('columns'))) 
col_diff

In [ ]:
row_diff = np.sqrt(np.square(src_ds.longitude.diff('rows')) + np.square(src_ds.latitude.diff('rows'))) 
row_diff

In [ ]:
col_diff.plot.imshow(figsize=(10,10), vmin=0.0, vmax=0.01)

In [ ]:
row_diff.plot.imshow(figsize=(10,10), vmin=0.003, vmax=0.005)

In [ ]:
t1 = time.perf_counter()
dst_ds = reproject_dataset(src_ds, var_names='Oa13_radiance', xy_names=('longitude', 'latitude'))
t2 = time.perf_counter()

In [ ]:
t2 - t1

In [ ]:
dst_ds

In [ ]:
dst_ds.Oa13_radiance.plot.imshow(figsize=(16,16), vmax=40, cmap='Greys_r')

In [ ]:
t1 = time.perf_counter()
dst_ds_dask = reproject_dataset(src_ds, var_names='Oa13_radiance', xy_names=('longitude', 'latitude'), tile_size=(1024, 2048))
t2 = time.perf_counter()

In [ ]:
t2 - t1

In [ ]:
dst_ds_dask

In [ ]:
dst_ds_dask.Oa13_radiance.data.visualize()

In [ ]:
t1 = time.perf_counter()
Oa13_radiance = dst_ds_dask.Oa13_radiance.values
t2 = time.perf_counter()

In [ ]:
t2 - t1

In [ ]:
dst_ds_dask.Oa13_radiance.plot.imshow(figsize=(16,16), vmax=40, cmap='Greys_r')

## Creating subsets from source `longitude` and `latitude` using a bounding box `bbox`

Bounding box `bbox` in decimal degree:

In [ ]:
bbox = (5, 62, 7, 64)

Compute `src_bbox` by selecting pixels in `bbox`:

In [ ]:
src_bbox = np.logical_and(np.logical_and(src_ds.longitude >= bbox[0], src_ds.longitude <= bbox[2]), 
                          np.logical_and(src_ds.latitude >= bbox[1], src_ds.latitude <= bbox[3]))
src_bbox.plot.imshow()

Select pixel coordinates from coordinate variables `x`, `y` using selected pixels in `src_bbox`:

In [ ]:
x = src_ds.columns.where(src_bbox)
y = src_ds.rows.where(src_bbox)

In [ ]:
x.plot.imshow()

In [ ]:
y.plot.imshow()

Get new bouning box `(x1, y1, x2, y2)` in pixel coordinates from min/max of `x` and `y`:

In [ ]:
x1 = int(x.min())
x2 = int(x.max())
y1 = int(y.min())
y2 = int(y.max())

In [ ]:
x1, y1, x2, y2

Use bounding box `(x1, y1, x2, y2)` to select subset within source image:

In [ ]:
rad = src_ds.Oa13_radiance.isel(columns=slice(x1, x2 + 1), rows=slice(y1, y2 + 1))
rad

In [ ]:
rad.plot.imshow(figsize=(10,10), vmax=40, cmap='Greys_r')

Verify: use bounding box `bbox` to select subset within target image:

In [ ]:
rad = dst_ds.Oa13_radiance.sel(longitude=slice(bbox[0], bbox[2]), latitude=slice(bbox[1], bbox[3]))
rad

In [ ]:
rad.plot.imshow(figsize=(10,10), vmax=40, cmap='Greys_r')